In [2]:
import langchain
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\junu\anaconda3\envs\chat\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3,google_api_key="AIzaSyba")
response=llm.invoke("who is the PM of India")

print(response.content)

Narendra Modi


In [13]:

from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings


class StorePdfData:
    def __init__(self,pdf_files,api_key) -> None:
        self.pdf_files=pdf_files
        self.api_key=api_key
    

    def extract_pdf_data(self):
        text=""
        for pdf in self.pdf_files:
            reader=PdfReader(pdf)
            for page in reader.pages:
                text += page.extract_text()
        return text
    
    def get_text_chunks(self,text):
        splitter=RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=0)
        chunks=splitter.split_text(text=text)
        return chunks
    
    def store_data_in_vector_store(self,chunks):
        embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=self.api_key)
        vector_stores=FAISS.from_texts(chunks,embeddings)
        vector_stores.save_local('faiss_index')



    
            



In [14]:
strdata=StorePdfData(["story.pdf"],api_key="AIzo")
text=strdata.extract_pdf_data()
chunks=strdata.get_text_chunks(text=text)
strdata.store_data_in_vector_store(chunks=chunks)

In [15]:
# db.save_local("faiss_index")
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key="AIz")
new_db = FAISS.load_local("faiss_index", embeddings)

docs = new_db.similarity_search("When Ferrars Died?")

In [16]:
docs

[Document(page_content="well together. After her husband's death, the intimacy became more marked.\nThey were always seen about together, and it was freely conjectured that at\nthe end of her period of mourning, Mrs Ferrars would become Mrs Roger"),
 Document(page_content="mystery of Mrs Ferrars's death. Had she taken her own life? Surely, if she\nhad done so, she would have left some word behind to say what she\ncontemplated doing? Women, in my experience, if they once reach the"),
 Document(page_content="CHAPTER 1. Dr Sheppard at the\nBreakfast Table\nMrs Ferrars died on the night of the 16th17th September - a Thursday. I\nwas sent for at eight o'clock on the morning of Friday the 17th. There was\nnothing to be done. She had been dead some hours."),
 Document(page_content='wife poisoned him.\nShe scorns my invariable rejoinder that Mr Ferrars died of acute\ngastritis, helped on by habitual overindulgence in alcoholic beverages. The\nsymptoms of gastritis and arsenical poisoning are n

In [12]:
chunks

["CHAPTER 1. Dr Sheppard at the\nBreakfast Table\nMrs Ferrars died on the night of the 16th17th September - a Thursday. I\nwas sent for at eight o'clock on the morning of Friday the 17th. There was\nnothing to be done. She had been dead some hours.\nIt was just a few minutes after nine when I reached home once more. I\nopened the front door with my latchkey, and purposely delayed a few\nmoments in the hall, hanging up my hat and the light overcoat that I had\ndeemed a wise precaution against the chill of an early autumn morning. To\ntell the truth, I was considerably upset and worried. I am not going to pretend\nthat at that moment I foresaw the events of the next few weeks. I\nemphatically did not do so.\nBut my instinct told me that there were stirring times ahead.\nFrom the dining-room on my left there came the rattle of tea-cups and\nthe short, dry cough of my sister Caroline.\n'Is that you, James?' she called.\nAn unnecessary question, since who else could it be? To tell the truth

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import ChatGoogleGenerativeAI



class ChatWithGemini:
    def __init__(self,api_key) -> None:
        self.api_key=api_key
        self.faiss_index="faiss_index"
    
    def question_search(self,question):

        embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=self.api_key)
        new_db = FAISS.load_local("faiss_index", embeddings)
        similar_docs=new_db.similarity_search(question)
        return similar_docs
    
    def question_search_in_gemini(self,docs,question):
        prompt=PromptTemplate(input_variables=["context","question"],
                              template="""Please read the below context \n
                              {context} \n
                              and answer this question {question} .\n
                              answer the question only if the required information available in above context. if it is not available then you can easily say ,
                                required information is not available in your PDF.
                                """)
        # prompt_temp=prompt.format(context=docs,question=question)
        llm=ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3,google_api_key=self.api_key)
        chain=load_qa_chain(llm,chain_type="stuff",prompt=prompt)
        # answer=chain.run(prompt_temp,)
        response=chain({"input_documents":docs,"question":question},return_only_outputs=True)
        return response
        # prompt.format()
        
        

In [18]:
chat=ChatWithGemini(api_key="AIzo")
docs=chat.question_search("When Ferrars Died?")
chat.question_search_in_gemini(docs=docs,question="When Ferrars Died?")

{'output_text': 'Mrs Ferrars died on the night of the 16th-17th September - a Thursday.'}